In [38]:
import tensorflow as tf
import time


g = tf.Graph()
with g.as_default():
    #a=["A","B","C","D"]
    #b=["1","2","3","4"]
    a = tf.random_uniform([1000, 100])
    b = tf.random_uniform([1000, 100])
    def model(a,b):
        #print("In Model GPU: ",i)
        print("In Model GPU A: ",a)
        print("In Model GPU B: ",b)
        c=a+b
        return c#a+b #"In GPU:"#+str(a)+" "+str(b)#+str(i)

    
    def make_parallel(fn, num_gpus, **kwargs):


        in_splits = {}
        for k, v in kwargs.items():
            in_splits[k] = tf.split(v, num_gpus)
        print("make_parallel in_splits", in_splits)
        out_split = []
        for i in range(num_gpus):
            print(i)
            #with tf.device(tf.DeviceSpec(device_type="CPU", device_index=i)):
                #with tf.variable_scope(tf.get_variable_scope(), reuse=i > 0):
            print("fff",{k : v[i] for k, v in in_splits.items()})
            out_split.append(fn(**{k : v[i] for k, v in in_splits.items()}))
        print("make_parallel out_split",out_split)
        return tf.concat(out_split, axis=0)
    c = make_parallel(model, 5, a=a, b=b)
    tf.summary.FileWriter("/home/fsg/logs_tf", g).close()


#with tf.device(tf.DeviceSpec(device_type="CPU", device_index=0)):
#a = tf.random_uniform([1000, 100])
#b = tf.random_uniform([1000, 100])
    #c = a + b



config = tf.ConfigProto(device_count={"GPU": 2,"CPU":1},
                        allow_soft_placement=True,
                        inter_op_parallelism_threads=16,
                        intra_op_parallelism_threads=16,
                        use_per_session_threads=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True
print("Start Time",time.time(), time.clock())

sess = tf.Session(config=config,graph=g)
#sess.run(tf.global_variables_initializer())

#print("C",sess.run(c))
#tf.summary.FileWriter("/home/fsg/logs_tf", g).close()


print("End Time",time.time(), time.clock())

make_parallel in_splits {'a': [<tf.Tensor 'split:0' shape=(200, 100) dtype=float32>, <tf.Tensor 'split:1' shape=(200, 100) dtype=float32>, <tf.Tensor 'split:2' shape=(200, 100) dtype=float32>, <tf.Tensor 'split:3' shape=(200, 100) dtype=float32>, <tf.Tensor 'split:4' shape=(200, 100) dtype=float32>], 'b': [<tf.Tensor 'split_1:0' shape=(200, 100) dtype=float32>, <tf.Tensor 'split_1:1' shape=(200, 100) dtype=float32>, <tf.Tensor 'split_1:2' shape=(200, 100) dtype=float32>, <tf.Tensor 'split_1:3' shape=(200, 100) dtype=float32>, <tf.Tensor 'split_1:4' shape=(200, 100) dtype=float32>]}
0
fff {'a': <tf.Tensor 'split:0' shape=(200, 100) dtype=float32>, 'b': <tf.Tensor 'split_1:0' shape=(200, 100) dtype=float32>}
In Model GPU A:  Tensor("split:0", shape=(200, 100), dtype=float32)
In Model GPU B:  Tensor("split_1:0", shape=(200, 100), dtype=float32)
1
fff {'a': <tf.Tensor 'split:1' shape=(200, 100) dtype=float32>, 'b': <tf.Tensor 'split_1:1' shape=(200, 100) dtype=float32>}
In Model GPU A:  Te